# 📝 管理Questbook任务（标准结构）

查看、添加、编辑任务

In [ ]:
import json
import gzip
import pandas as pd
from pathlib import Path
from datetime import datetime
from IPython.display import display

print("✓ 模块加载成功")

## ⚙️ 选择版本

In [ ]:
VERSION = "custom"  # 修改为你的版本代号

BASE = Path("..") / "bin" / "version" / VERSION
ICONS = BASE / "quests_icons"

if not BASE.exists():
    print(f"❌ 版本不存在: {VERSION}")
else:
    # 加载数据
    with open(BASE / "quest_json.json", encoding='utf-8') as f:
        data_zh = json.load(f)
    with open(BASE / "quest_json_en.json", encoding='utf-8') as f:
        data_en = json.load(f)
    with open(BASE / "quest_line.json", encoding='utf-8') as f:
        lines = json.load(f)
    with open(BASE / "quests_icons.json", encoding='utf-8') as f:
        atlas = json.load(f)
    
    print(f"✓ 版本: {VERSION}")
    print(f"  任务线: {len(lines)}")
    print(f"  任务总数: {sum(len(v['data']) for v in data_zh.values())}")

## 📊 查看所有任务

In [ ]:
def show_quests():
    rows = []
    for line, ld in data_zh.items():
        for q in ld['data']:
            prereqs = [lk['source'] for lk in ld['links'] if lk['target'] == q['name']]
            rows.append({
                '任务线': line, '任务名': q['name'], '标题': q['title'],
                '坐标': f"({q['x']},{q['y']})", '大小': q['symbolSize'],
                '类型': '主线' if q['is_main'] else '支线',
                '前置': ','.join(prereqs) if prereqs else '无',
                'ID': q['quest_id']
            })
    return pd.DataFrame(rows)

df = show_quests()
print(f"\n共 {len(df)} 个任务:\n")
display(df)

## ➕ 添加新任务

In [ ]:
# ========== 新任务配置 ==========
NEW = {
    "questline": "WelcomeQuests",  # 必须已存在
    "name": "ThirdQuest",
    "title_zh": "§b§l第三个任务§r",
    "title_en": "§b§lThird Quest§r",
    "desc_zh": "§e继续冒险§r",
    "desc_en": "§eContinue§r",
    "x": 500, "y": 150, "size": 50,
    "is_main": 1,
    "prereqs": ["GettingStarted"]
}

print(f"任务: {NEW['name']} → {NEW['title_zh']}")

In [ ]:
line = NEW['questline']
if line not in data_zh:
    print(f"❌ 任务线不存在: {line}")
else:
    # 检查重复
    names = [q['name'] for q in data_zh[line]['data']]
    if NEW['name'] in names:
        print(f"❌ 任务名已存在: {NEW['name']}")
    else:
        # 生成ID
        qid = f"{NEW['name'].lower()}_{int(datetime.now().timestamp())}"
        
        # 添加节点（中文）
        data_zh[line]['data'].append({
            "name": NEW['name'], "symbolSize": NEW['size'],
            "symbol": "", "x": NEW['x'], "y": NEW['y'],
            "data": NEW['desc_zh'], "quest_id": qid,
            "title": NEW['title_zh'], "is_main": NEW['is_main'],
            "tooltip": {"show": True}
        })
        
        # 添加节点（英文）
        data_en[line]['data'].append({
            "name": NEW['name'], "symbolSize": NEW['size'],
            "symbol": "", "x": NEW['x'], "y": NEW['y'],
            "data": NEW['desc_en'], "quest_id": qid,
            "title": NEW['title_en'], "is_main": NEW['is_main'],
            "tooltip": {"show": True}
        })
        
        # 添加前置关系
        for prereq in NEW['prereqs']:
            if prereq not in names:
                print(f"⚠️ 前置任务不存在: {prereq}")
                continue
            link = {
                "source": prereq, "target": NEW['name'],
                "symbol": ["none", "arrow"],
                "lineStyle": {
                    "color": "#00c800" if NEW['is_main'] else "#666666",
                    "width": 4 if NEW['is_main'] else 2
                },
                "symbolSize": [0, 10]
            }
            data_zh[line]['links'].append(link)
            data_en[line]['links'].append(link)
        
        # 更新图集
        atlas_key = f"QuestIcon/{line}"
        if atlas_key not in atlas:
            atlas[atlas_key] = []
        atlas[atlas_key].append(qid)
        
        # 添加占位符图标
        DUMMY = "iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAYAAAAfFcSJAAAADUlEQVR42mNk+M9QDwADhgGAWjR9awAAAABJRU5ErkJggg=="
        icon_file = ICONS / "QuestIcon" / f"{line}.json"
        
        if icon_file.exists():
            with open(icon_file, encoding='utf-8') as f:
                icons = json.load(f)
        else:
            icons = {}
        
        icons[qid] = DUMMY
        
        with open(icon_file, 'w', encoding='utf-8') as f:
            json.dump(icons, f, ensure_ascii=False, indent=2)
        
        with gzip.open(icon_file.with_suffix('.gtbl'), 'wb') as f:
            f.write(json.dumps(icons, ensure_ascii=False).encode('utf-8'))
        
        print(f"✓ 任务已添加: {NEW['name']} (ID: {qid})")
        print("\n📊 更新后的任务:")
        display(show_quests())

## 💾 保存所有更改

In [ ]:
# 保存任务数据
with open(BASE / "quest_json.json", 'w', encoding='utf-8') as f:
    json.dump(data_zh, f, ensure_ascii=False, indent=2)
print("✓ quest_json.json")

with open(BASE / "quest_json_en.json", 'w', encoding='utf-8') as f:
    json.dump(data_en, f, ensure_ascii=False, indent=2)
print("✓ quest_json_en.json")

# 保存图集
with open(BASE / "quests_icons.json", 'w', encoding='utf-8') as f:
    json.dump(atlas, f, ensure_ascii=False, indent=2)
print("✓ quests_icons.json")

print("\n✨ 所有更改已保存！")
print("\n📌 后续步骤:")
print("  1. Commit 到 GitHub")
print("  2. 等待自动构建")
print("  3. 或本地测试: npm run build")

## 📈 统计

In [ ]:
total = sum(len(v['data']) for v in data_zh.values())
main = sum(sum(1 for q in v['data'] if q['is_main']) for v in data_zh.values())
links = sum(len(v['links']) for v in data_zh.values())

print("="*50)
print(f"📊 {VERSION} 统计")
print("="*50)
print(f"任务线: {len(data_zh)}")
print(f"任务总数: {total}")
print(f"  主线: {main} | 支线: {total-main}")
print(f"前置关系: {links}")
print("="*50)